In [66]:
import requests
import random
import logging
import time
import csv
import pandas as pd
import os
import io

DATA_DIR_RAW = os.path.join("..", "..", "data", "raw")
DATA_DIR_PROCESSED = os.path.join("..", "..", "data", "processed")

# 🪵 Logging setup
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

In [58]:
def write_to_csv(data, output_file, replace=False):
    if os.path.exists(output_file) and not replace:
        logging.info(f"☑️ File '{output_file}' already exists. Skipping write.")
        return

    try:
        if isinstance(data, pd.DataFrame):
            # Handle DataFrame directly
            if data.empty:
                logging.warning("⚠️ No data to write (DataFrame is empty).")
                return
            data.to_csv(output_file, index=False, encoding="utf-8")
            logging.info(f"📄 Successfully wrote {len(data)} rows (DataFrame) to '{output_file}'")

        elif isinstance(data, list):
            # Handle list of dicts
            if not data:
                logging.warning("⚠️ No data to write (list is empty).")
                return
            fieldnames = sorted(set().union(*(d.keys() for d in data)))
            with open(output_file, mode="w", newline="", encoding="utf-8") as f:
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                writer.writeheader()
                writer.writerows(data)
            logging.info(f"📄 Successfully wrote {len(data)} rows (list of dicts) to '{output_file}'")

        else:
            logging.error(f"❌ Unsupported data type: {type(data)}. Expected DataFrame or list of dicts.")

    except Exception as e:
        logging.error(f"❌ Failed to write CSV: {e}")

In [38]:
ACE_endpoint_url = "https://data.ny.gov/resource/kh8p-hcbm.csv"
CBD_endpoint_url = "https://data.ny.gov/resource/cgzt-smqf.csv"


In [19]:
# fetching CBD_bus_routes
CBD_bus_routes = requests.get(url=CBD_endpoint_url)
CBD_bus_routes = pd.read_csv(io.StringIO(CBD_bus_routes.text))
CBD_bus_routes.head()

,route_id,cbd_relation
0,BM1,Crossing CBD
1,BM2,Crossing CBD
2,BM3,Crossing CBD
3,BM4,Crossing CBD
4,BM5,Crossing CBD


In [25]:
OUTPUT_CBD = os.path.join(DATA_DIR_RAW, "CBD_bus_routes.csv")
write_to_csv(CBD_bus_routes, OUTPUT_CBD)

2025-09-20 23:14:04,727 - INFO - ☑️ File '..\..\data\raw\CBD_bus_routes.csv' already exists. Skipping write.


In [32]:
route_ids = CBD_bus_routes['route_id'].unique()
route_list = "', '".join(route_ids)
print(f"""'{route_list}'""")

'BM1', 'BM2', 'BM3', 'BM4', 'BM5', 'BXM1', 'BXM10', 'BXM11', 'BXM18', 'BXM2', 'BXM3', 'BXM4', 'BXM6', 'BXM7', 'BXM8', 'BXM9', 'QM1', 'QM10', 'QM11', 'QM12', 'QM15', 'QM16', 'QM17', 'QM18', 'QM2', 'QM20', 'QM21', 'QM24', 'QM25', 'QM3', 'QM31', 'QM32', 'QM34', 'QM35', 'QM36', 'QM4', 'QM40', 'QM42', 'QM44', 'QM5', 'QM6', 'QM7', 'QM8', 'SIM1', 'SIM10', 'SIM11', 'SIM15', 'SIM2', 'SIM22', 'SIM23', 'SIM24', 'SIM25', 'SIM26', 'SIM3', 'SIM30', 'SIM31', 'SIM32', 'SIM33', 'SIM34', 'SIM35', 'SIM4', 'SIM5', 'SIM6', 'SIM7', 'SIM8', 'SIM9', 'X27', 'X28', 'X37', 'X38', 'X63', 'X64', 'X68', 'B39', 'M1', 'M10', 'M101', 'M102', 'M103', 'M104', 'M11', 'M12', 'M14A+', 'M14D+', 'M15', 'M15+', 'M2', 'M20', 'M21', 'M22', 'M23+', 'M3', 'M31', 'M34+', 'M34A+', 'M4', 'M42', 'M5', 'M50', 'M55', 'M57', 'M7', 'M8', 'M9', 'Q32', 'Q60', 'Q101'


In [51]:
all_data = []
limit = 50000
offset = 0

"""
Find the number of rows in dataset
then round to the nearest million or billion whichever
you can use to make sure you get all data
"""
max_rows = 3800000 #rounding to the nearest hundreds place

while offset < max_rows:
    print(f"Fetching rows {offset + 1} to {offset + limit}")

    soql_query = f"SELECT * " \
             f"WHERE bus_route_id IN ('{route_list}') " \
             f"LIMIT {limit} " \
             f"OFFSET {offset}"

    response = requests.get(
        url=ACE_endpoint_url,
        params={
            "$query": soql_query
        }
    )

    # Convert to DataFrame
    chunk_data = pd.read_csv(io.StringIO(response.text))

    all_data.append(chunk_data)

    offset += limit

    # Respect 1-second crawl delay before next request
    time.sleep(1)

# Combine all chunks into one DataFrame
non_empty_data = [df for df in all_data if len(df) > 0]
CBD_ACE_violations = pd.concat(non_empty_data, ignore_index=True)
print(f"Total rows: {len(CBD_ACE_violations)}")

#in case of automation
time.sleep(1)

Fetching rows 1 to 50000
Fetching rows 50001 to 100000
Fetching rows 100001 to 150000
Fetching rows 150001 to 200000
Fetching rows 200001 to 250000
Fetching rows 250001 to 300000
Fetching rows 300001 to 350000
Fetching rows 350001 to 400000
Fetching rows 400001 to 450000
Fetching rows 450001 to 500000
Fetching rows 500001 to 550000
Fetching rows 550001 to 600000
Fetching rows 600001 to 650000
Fetching rows 650001 to 700000
Fetching rows 700001 to 750000
Fetching rows 750001 to 800000
Fetching rows 800001 to 850000
Fetching rows 850001 to 900000
Fetching rows 900001 to 950000
Fetching rows 950001 to 1000000
Fetching rows 1000001 to 1050000
Fetching rows 1050001 to 1100000
Fetching rows 1100001 to 1150000
Fetching rows 1150001 to 1200000
Fetching rows 1200001 to 1250000
Fetching rows 1250001 to 1300000
Fetching rows 1300001 to 1350000
Fetching rows 1350001 to 1400000
Fetching rows 1400001 to 1450000
Fetching rows 1450001 to 1500000
Fetching rows 1500001 to 1550000
Fetching rows 1550001 t

In [52]:
CBD_ACE_violations.head()

,violation_id,vehicle_id,first_occurrence,last_occurrence,violation_status,violation_type,bus_route_id,violation_latitude,violation_longitude,stop_id,stop_name,bus_stop_latitude,bus_stop_longitude,violation_georeference,bus_stop_georeference
0,489741940,7feac037b62d591ffb1214e356157f3dd197fc22fee5bb...,2025-08-20T10:52:57.000,2025-08-20T11:16:57.000,EXEMPT - EMERGENCY VEHICLE,MOBILE BUS STOP,M101,40.815113,-73.955040,401458,AMSTERDAM AV/W 131 ST,40.816009,-73.954424,POINT (-73.95504 40.815113),POINT (-73.954424 40.816009)
1,489741935,e9122b4e6dac9160ed4fca952a3815fe4c737bfc70cb97...,2025-08-20T23:35:29.000,2025-08-20T23:39:32.000,EXEMPT - EMERGENCY VEHICLE,MOBILE BUS STOP,M101,40.795630,-73.941683,402704,3 AV/E 109 ST,40.793765,-73.943005,POINT (-73.941683 40.79563),POINT (-73.943005 40.793765)
2,489741747,de04d0589e26e7da94c192dff0be14d105abcc8342c87f...,2025-08-20T23:18:06.000,2025-08-20T23:24:10.000,EXEMPT - EMERGENCY VEHICLE,MOBILE BUS LANE,M101,40.808403,-73.946993,402658,W 125 ST/ADAM C POWELL BLVD,40.808700,-73.947836,POINT (-73.946993 40.808403),POINT (-73.947836 40.8087)
3,489741639,6d7caef8bf15e2bf8d46aa1e6dd8c0da1ae2fbc5b31ace...,2025-08-20T22:58:39.000,2025-08-20T23:02:18.000,EXEMPT - EMERGENCY VEHICLE,MOBILE DOUBLE PARKED,M101,40.842800,-73.934798,405069,AMSTERDAM AV/W 175 ST,40.843810,-73.934076,POINT (-73.934798 40.8428),POINT (-73.934076 40.84381)
4,489741496,414bf604b556873481cb6a6fa325cef72d37bf7c168385...,2025-08-21T12:01:22.000,2025-08-21T14:40:41.000,EXEMPT - EMERGENCY VEHICLE,MOBILE BUS STOP,M42,40.747781,-73.969323,903046,E 41 ST/1 AV,40.747979,-73.969789,POINT (-73.96932254145938 40.74778115789982),POINT (-73.969789 40.747979)


In [59]:
OUTPUT_CBD_ACE = os.path.join(DATA_DIR_RAW, "CBD_ACE_violations.csv")
write_to_csv(CBD_ACE_violations, OUTPUT_CBD_ACE)

2025-09-21 00:38:43,573 - INFO - 📄 Successfully wrote 898239 rows (DataFrame) to '..\..\data\raw\CBD_ACE_violations.csv'


In [67]:
# Merge CBD_ACE_violations and CBD_bus_routes
CBD_violations_merged = CBD_ACE_violations.merge(
    CBD_bus_routes,
    left_on='bus_route_id',
    right_on='route_id',
    how='inner' #CBD_bus_routes was used to generate CBD_ACE_violations so this doesn't matter
)

CBD_violations_merged = CBD_violations_merged.drop(columns=['route_id'])

OUTPUT_merge = os.path.join(DATA_DIR_PROCESSED, "CBD_violations_merged.csv")
write_to_csv(CBD_violations_merged, OUTPUT_merge)
CBD_violations_merged.head()

2025-09-21 01:05:44,303 - INFO - 📄 Successfully wrote 898239 rows (DataFrame) to '..\..\data\processed\CBD_violations_merged.csv'


,violation_id,vehicle_id,first_occurrence,last_occurrence,violation_status,violation_type,bus_route_id,violation_latitude,violation_longitude,stop_id,stop_name,bus_stop_latitude,bus_stop_longitude,violation_georeference,bus_stop_georeference,cbd_relation
0,489741940,7feac037b62d591ffb1214e356157f3dd197fc22fee5bb...,2025-08-20T10:52:57.000,2025-08-20T11:16:57.000,EXEMPT - EMERGENCY VEHICLE,MOBILE BUS STOP,M101,40.815113,-73.955040,401458,AMSTERDAM AV/W 131 ST,40.816009,-73.954424,POINT (-73.95504 40.815113),POINT (-73.954424 40.816009),Crossing CBD
1,489741935,e9122b4e6dac9160ed4fca952a3815fe4c737bfc70cb97...,2025-08-20T23:35:29.000,2025-08-20T23:39:32.000,EXEMPT - EMERGENCY VEHICLE,MOBILE BUS STOP,M101,40.795630,-73.941683,402704,3 AV/E 109 ST,40.793765,-73.943005,POINT (-73.941683 40.79563),POINT (-73.943005 40.793765),Crossing CBD
2,489741747,de04d0589e26e7da94c192dff0be14d105abcc8342c87f...,2025-08-20T23:18:06.000,2025-08-20T23:24:10.000,EXEMPT - EMERGENCY VEHICLE,MOBILE BUS LANE,M101,40.808403,-73.946993,402658,W 125 ST/ADAM C POWELL BLVD,40.808700,-73.947836,POINT (-73.946993 40.808403),POINT (-73.947836 40.8087),Crossing CBD
3,489741639,6d7caef8bf15e2bf8d46aa1e6dd8c0da1ae2fbc5b31ace...,2025-08-20T22:58:39.000,2025-08-20T23:02:18.000,EXEMPT - EMERGENCY VEHICLE,MOBILE DOUBLE PARKED,M101,40.842800,-73.934798,405069,AMSTERDAM AV/W 175 ST,40.843810,-73.934076,POINT (-73.934798 40.8428),POINT (-73.934076 40.84381),Crossing CBD
4,489741496,414bf604b556873481cb6a6fa325cef72d37bf7c168385...,2025-08-21T12:01:22.000,2025-08-21T14:40:41.000,EXEMPT - EMERGENCY VEHICLE,MOBILE BUS STOP,M42,40.747781,-73.969323,903046,E 41 ST/1 AV,40.747979,-73.969789,POINT (-73.96932254145938 40.74778115789982),POINT (-73.969789 40.747979),In CBD
